<img src='pic/logo.png'/>

   # DJANGO-REST-FRAMEWORK TUTORIAL
   
   ## Majid Iranpour
   ## Twitter: @_majidmc2
   <br><br><br>
   <hr>
   <br><br><br>

# Authentication

## Create superuser:
> python manage.py createsuperuser

## TokenAuthentication:

### settings.py

In [ ]:
INSTALLED_APPS = (
    ...
    'rest_framework.authtoken',
)

 >   run "python manage.py migrate" after changing your settings

In [ ]:
REST_FRAMEWORK = {
   'DEFAULT_AUTHENTICATION_CLASSES': (
       'rest_framework.authentication.TokenAuthentication',
   ),
   'DEFAULT_PERMISSION_CLASSES': ('rest_framework.permissions.IsAdminUser',
                       'rest_framework.permissions.IsAuthenticated'),
}

### urls.py

In [ ]:
from rest_framework.authtoken import views

urlpatterns = [
    path('api-token-auth', views.obtain_auth_token),
    path('api-auth', include('rest_framework.urls', namespace='rest_framework')),  # UI Page
    ...
]

<img src='pic/api-token-auth.png'/>

<img src='pic/request-with-token.png'/>

## SessionAuthentication:

### settings.py

In [ ]:
REST_FRAMEWORK = {
    'DEFAULT_AUTHENTICATION_CLASSES': (
        'rest_framework.authentication.SessionAuthentication',
        ...
    ),
    ...
}

# Filter Backends &  Pagination



>  Run "pip install django-filter" 
### settings.py

In [ ]:
INSTALLED_APPS = (
    ...
    'django_filters',
)

In [ ]:
REST_FRAMEWORK = {
    
    ...
    
    'DEFAULT_FILTER_BACKENDS': ('django_filters.rest_framework.DjangoFilterBackend',
                                'rest_framework.filters.SearchFilter',
                                'rest_framework.filters.OrderingFilter'),

    'DEFAULT_PAGINATION_CLASS': 'rest_framework.pagination.LimitOffsetPagination',
    'PAGE_SIZE': 3

}

### views.py

In [ ]:
class BookViewSet(viewsets.ModelViewSet):
    queryset = Book.objects.all()
    serializer_class = BookSerializer
    http_method_names = ['get', 'post', 'put', 'patch', 'delete']

    search_fields = ('pages', 'name')
    ordering_fields = '__all__'
    
    ....

<img src='pic/ordering-search.png'/>

# Create Endpoint User

### serializers.py

In [ ]:
from django.contrib.auth.models import User


class UserSerializer(serializers.ModelSerializer):
    password = serializers.CharField(write_only=True)

    class Meta:
        model = User
        fields = '__all__'

    def create(self, validated_data):
        user = super(UserSerializer, self).create(validated_data)
        user.set_password(validated_data['password'])
        user.save()
        return user

### views.py

In [ ]:
from rest_framework.permissions import AllowAny

@api_view(['POST'])
@permission_classes((AllowAny,))
def user(request):
    serialized = UserSerializer(data=request.data)
    if serialized.is_valid():
        serialized.save()
        return Response(status=status.HTTP_201_CREATED)
    else:
        return Response(serialized.errors, status=status.HTTP_400_BAD_REQUEST)

### urls.py

In [ ]:
from django.urls import path


urlpatterns = [
    ...
    path(r'user', views.user, name='user'),
]

# Permissions

> Each user only can view its informations

### views.py

In [ ]:
from rest_framework import viewsets

from rest_framework.permissions import IsAuthenticated
from rest_framework.decorators import api_view, permission_classes
from rest_framework.response import Response
from rest_framework import status
from django.contrib.auth.models import User

from .permissions import IsOwner
from .serializers import UserSerializer


@api_view(['POST'])
@permission_classes((IsAuthenticated, IsOwner))
def get_user(request):
    try:
        employ = User.objects.get(username=request.data['username'])
    except User.DoesNotExist:
        return Response(status=status.HTTP_404_NOT_FOUND)

    serializer = UserSerializer(employ)
    return Response(serializer.data, status=status.HTTP_200_OK)


### permissions.py

In [ ]:
from rest_framework import permissions
from django.contrib.auth.models import User


class IsOwner(permissions.BasePermission):

    def has_permission(self, request, view):
        try:
            user = User.objects.get(username=request.data['username'])
        except User.DoesNotExist:
            return False

        if request.user.is_superuser:
            return True

        if not request.user == user:
            return False

        return True


### urls.py

In [ ]:
from django.urls import path


urlpatterns = [
    ...
    path(r'get_user', views.get_user, name='get_user'),
]


# Disable DEBUG mod


In [ ]:
DEBUG = False

ALLOWED_HOSTS = ['127.0.0.1']


.
.
.


DEFAULT_RENDERER_CLASSES = (
    'rest_framework.renderers.JSONRenderer',
)

if DEBUG:
    DEFAULT_RENDERER_CLASSES = DEFAULT_RENDERER_CLASSES + (
        'rest_framework.renderers.BrowsableAPIRenderer',
    )

REST_FRAMEWORK = {
    'DEFAULT_PERMISSION_CLASSES': (
        'rest_framework.permissions.IsAuthenticatedOrReadOnly',
    ),
    'DEFAULT_RENDERER_CLASSES': DEFAULT_RENDERER_CLASSES
}


# Connect Djang0 to MySQL

### settings.py

In [ ]:
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.mysql',
        'OPTIONS': {
            'read_default_file': '/etc/mysql/my.cnf',
        },
    }
}

...

### my.cnf
> $ sudo nano /etc/mysql/my.cnf

In [ ]:
...

[client]
database = db_name
user = db_user
password = db_password
default-character-set = utf8